# Analyse de l'Amplitude Thermique Cumulative en Occitanie : Tendances et Insights


L'analyse présentée dans ce notebook se concentre sur l'exploration des données météorologiques quotidiennes de la région Occitanie, en particulier sur l'amplitude thermique journalière et sa somme cumulative annuelle. Les données utilisées proviennent du site data.gouv.fr et sont accessibles via l'URL : https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_34_previous-1950-2023_RR-T-Vent.csv.gz. Ce fichier contient des informations météorologiques détaillées relevées dans différentes stations de la région, couvrant une période allant de 1950 à 2023.


L'objectif principal de cette analyse est de calculer la somme cumulative de l'amplitude thermique journalière pour chaque année, en moyennant d'abord sur toutes les stations, et en ne gardant que la valeur de fin de chaque année. Cette approche permet d'identifier les tendances et les variations dans l'amplitude thermique cumulative sur le long terme, offrant ainsi des insights précieux pour comprendre les dynamiques climatiques en Occitanie.


## Méthodologie

La méthodologie adoptée pour cette analyse implique plusieurs étapes clés. Tout d'abord, les données sont chargées et nettoyées pour préparer l'analyse. Ensuite, une requête SQL est exécutée via DuckDB pour calculer l'amplitude thermique journalière moyenne par jour sur toutes les stations, puis pour déterminer la somme cumulative de cette amplitude pour chaque année. Les résultats sont ensuite filtrés pour ne conserver que la dernière valeur de chaque année. Enfin, les données résultantes sont visualisées à l'aide de Plotly pour mettre en évidence les tendances et les points notables dans l'amplitude thermique cumulative.


Les résultats de cette analyse sont présentés sous forme de graphiques interactifs, permettant une exploration détaillée des tendances et des variations dans l'amplitude thermique cumulative en Occitanie sur la période étudiée.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_34_previous-1950-2023_RR-T-Vent.csv.gz", safe_mode=True)
print("✅ Données chargées avec succès")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" WITH 
-- Calculer l'amplitude thermique journalière moyenne par jour sur toutes les stations
daily_avg AS (
    SELECT 
        "AAAAMMJJ", 
        AVG(CAST("TX" AS DOUBLE) - CAST("TN" AS DOUBLE)) AS avg_amplitude
    FROM 
        loaded_dataset
    GROUP BY 
        "AAAAMMJJ"
),
-- Extraire l'année et calculer la somme cumulative de l'amplitude thermique
cumulative_sum AS (
    SELECT 
        SUBSTRING(CAST("AAAAMMJJ" AS VARCHAR), 1, 4) AS year,
        "AAAAMMJJ",
        avg_amplitude,
        SUM(avg_amplitude) OVER (PARTITION BY SUBSTRING(CAST("AAAAMMJJ" AS VARCHAR), 1, 4) ORDER BY "AAAAMMJJ") AS cumulative_amplitude
    FROM 
        daily_avg
)
-- Ne garder que la dernière valeur de chaque année
SELECT 
    year, 
    "AAAAMMJJ", 
    cumulative_amplitude
FROM 
    cumulative_sum
WHERE 
    "AAAAMMJJ" IN (SELECT MAX("AAAAMMJJ") FROM cumulative_sum GROUP BY year)
ORDER BY 
    year """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 74 lignes


,year,AAAAMMJJ,cumulative_amplitude
0,1950,19501231,3777.683333
1,1951,19511231,3513.591667
2,1952,19521231,3597.383333
3,1953,19531231,3557.633333
4,1954,19541231,3709.083333


## 📈 Visualisation

La bibliothèque principale utilisée est Plotly, qui permet de créer des visualisations interactives et personnalisables. Cette technologie est adaptée pour représenter des données complexes avec plusieurs couches d'information, comme ici où l'on affiche des points de données brutes, une courbe brute et une courbe lissée avec des annotations. Plotly offre une grande flexibilité pour créer des graphiques attractifs et informatifs.

In [4]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from scipy.ndimage import gaussian_filter1d

# Nettoyage rapide
df['year'] = df['AAAAMMJJ'].astype(str).str[:4].astype(int)

# Création graphe
fig = go.Figure()

# 1. Points bruts rouge clair
fig.add_trace(go.Scatter(
    x=df['year'], 
    y=df['cumulative_amplitude'],
    mode='markers',
    name='Données brutes (fin d’année)',
    marker=dict(color='#FF9999', size=6, opacity=0.9)
))

# 2. Ligne brute gris très clair
fig.add_trace(go.Scatter(
    x=df['year'], 
    y=df['cumulative_amplitude'],
    mode='lines',
    name='Courbe brute',
    line=dict(color='#E5E5E5', width=1)
))

# 3. Lisse - rolling 5 ans précédents
roll = (
    pd.Series(df['cumulative_amplitude'])
      .rolling(window=5, center=True, min_periods=1)
      .mean()
).values

# Ajout du trace lissé aire bleue
fig.add_trace(go.Scatter(
    x=df['year'], 
    y=roll,
    fill='tonexty',
    mode='lines',
    name='Courbe lissée 5 ans',
    line=dict(color='#007acc', width=2.5),
    fillcolor='rgba(0,122,204,0.25)'
))

# Top 10 annotations avec tooltip personnalisé
top10 = df.nlargest(10, 'cumulative_amplitude')
for _, row in top10.iterrows():
    fig.add_annotation(
        x=row['year'],
        y=row['cumulative_amplitude'],
        text=str(row['year']),
        font=dict(size=9, color='black'),
        showarrow=True,
        arrowhead=2,
        arrowsize=0.7,
        arrowwidth=1,
        ax=0,
        ay=-30,
        align='center',
        bgcolor='#FFFFFF',
        bordercolor='#CCCCCC',
        borderwidth=0.5
    )

fig.update_layout(
    title=None,
    margin=dict(l=20, r=20, t=40, b=60),
    xaxis_title='Année',
    yaxis_title='Amplitude thermique cumulative (°C)',
    legend=dict(
        x=0.01, y=0.99,
        bgcolor='rgba(255,255,255,0.85)', bordercolor='LightGray', borderwidth=0.5
    ),
    template='plotly_white',
    height=450,
    xaxis=dict(dtick=5, showspikes=False),
    yaxis=dict(
        showspikes=False,
        gridcolor='#F5F5F5'
    ),
    plot_bgcolor='rgba(0,0,0,0)'
)

dataviz = fig
dataviz

ModuleNotFoundError: No module named 'scipy'

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250820_230227.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250820_230227.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


AVERTISSEMENT: Aucune variable 'dataviz' trouvée.
